# Derivatives

In [ ]:
import numpy as np
from itertools import product
import matplotlib.pylab as plt

# Finite difference dispersion

There is a one to one corespondence between a finite step derivative and the dispersion of the corresponding momentum operator on the lattice.

E.g. for a Laplace finite step derivative of the form

$$
  \frac{\partial^2}{\partial^2x}f(x) \mapsto
  \frac{1}{\epsilon^2}\sum_{n=-N_s}^{N_s} c_n^{N_s} f(\epsilon n_x + \epsilon n) \, ,
  x = \epsilon n_x \, ,
$$
where $\epsilon$ is the lattice spacing, corresponds to the momentum dispersion
$$
    p^2 \mapsto D^{N_s}(p) = - \frac{1}{\epsilon^2}\sum_{n=-N_s}^{N_s} c_n^{N_s} \cos(n p \epsilon) \, .
$$

In general, one wants to identify the coefficients $c_n^{N_s}$ such that $D^{N_s}(p) = p^2 + \mathcal O \left( (\epsilon p)^{2 N_s} \right)$. 


If one now expands 
$$
    \cos(n p \epsilon) = \sum_{m=0}^\infty \frac{(-)^m}{(2m)!} (n p \epsilon)^{2m}\, ,
$$
one finds that the coefficients $c_n$ are determined by the liner equation
\begin{align}
    v_m &= \sum_{n=0}^{N_s} A_{m n} \gamma_n
    \, , & 
    A_{mn} &= \frac{(-)^m}{(2m)!} n^{2m} \, ,
    \\
    v_1 &= 1 
    \, , & 
    v_m &= 0 \, \forall \, m \neq 1
    \\
    c_0 &= - \gamma_0 
    \, , &
    c_{\pm n} &= - \frac{\gamma_n}{2} \, \forall \, n > 0
\end{align}
where $n$ and $m$ run from $0$ to $N_s$.


In [ ]:
def derivative_coeffs(Nstep: int):
    """Computes the derivative coefficient for the lapace operator up to step range Nstep.
    
    The dispersion of the related momentum squared operator is equal to p**2 up to order P**(2*NStep).
    
    **Arguments**
        Nstep: int
            The number of momentum steps in each direction.
    """
    v = np.zeros(Nstep + 1)
    v[1] = 1

    A = np.zeros([Nstep + 1, Nstep + 1])
    for m, n in product(*[range(Nstep + 1)] * 2):
        A[m, n] = 1 / np.math.factorial(2 * m) * (-1) ** m * n ** (2 * m)

    gamma = np.linalg.solve(A, v)

    coeffs = {}
    for nstep, coeff in enumerate(gamma):
        if nstep == 0:
            coeffs[nstep] = -coeff
        else:
            coeffs[+nstep] = -coeff / 2
            coeffs[-nstep] = -coeff / 2

    return coeffs


In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), dpi=250)

p = np.linspace(-1, 1, 1000)

for nstep in range(2, 10, 2):
    coeffs = derivative_coeffs(nstep)
    Dp = np.sum([-cn * np.cos(n * p) for n, cn in coeffs.items()], axis=0)
    ax.plot(p, p ** 2 - Dp, label=f"$D^{{({nstep})}}(p)$", lw=1)

ax.legend(fontsize=6, frameon=False)
ax.set_xlabel("$p$")
ax.set_ylabel("$p^2 - D(p)$")
ax.set_yscale("log")
ax.set_ylim(1.0e-12, 1)

plt.show(fig)